# Домашнее задание 22: TF-IDF и Word2Vec на корпусе Helsinki-NLP/opus-100

## Установка зависимостей (при необходимости)
При первом запуске раскомментируйте команды ниже.

In [31]:
# !pip install datasets nltk pymorphy3 natasha gensim

## Импорт библиотек и подготовка инструментов
Подключаем всё необходимое для обработки текста и моделей.

In [32]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import pymorphy3
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from huggingface_hub import hf_hub_download


nltk.download('punkt')
nltk.download('stopwords')
morph = pymorphy3.MorphAnalyzer()
russian_stopwords = set(stopwords.words('russian'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\stepa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\stepa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Загрузка корпуса и разбиение на обучающую и тестовую части
Используем 1200 предложений: 1000 для обучения и 200 для проверки.

In [33]:
pair = "en-ru"
LANG_SRC, LANG_TGT = pair.split("-")

DATASET_REPO = "Helsinki-NLP/opus-100"

def load_split(split: str):
    filename = f"{LANG_SRC}-{LANG_TGT}/{split}-00000-of-00001.parquet"
    local_path = hf_hub_download(DATASET_REPO, filename=filename, repo_type="dataset")
    df = pd.read_parquet(local_path)
    return [{"translation": rec} for rec in df["translation"].tolist()]

train_ds = load_split("train")
val_ds   = load_split("validation")
test_ds  = load_split("test")

print(f"Размеры исходных сплитов: train={len(train_ds)}, val={len(val_ds)}, test={len(test_ds)}")

Размеры исходных сплитов: train=1000000, val=2000, test=2000


In [34]:
sample_size = 200000  # ограничим объём для отладки и анализа
train_subset = train_ds[:sample_size]
train_df = pd.DataFrame({
    'ru': [row['translation'][LANG_TGT] for row in train_subset],
    'en': [row['translation'][LANG_SRC] for row in train_subset],
})
train_df.head()

,ru,en
0,"Да, но не совсем...","Yeah, that's not exactly..."
1,!,!
2,Приводимое ниже расписание является предварите...,The schedule below is tentative; up-to-date in...
3,"Но сейчас ...я вверяю вам удостовериться, что ...","But for now,"
4,Они тусовались там несколько недель.,He'd been out there a few weeks or so.


In [35]:
test_subset = test_ds[:200]
test_df = pd.DataFrame({
    'ru': [row['translation'][LANG_TGT] for row in test_subset],
    'en': [row['translation'][LANG_SRC] for row in test_subset],
})

## Определяем функции предварительной обработки
Токенизация, приведение к нижнему регистру, лемматизация и очистка.

In [36]:
def preprocess_sentence(sentence):
    tokens = word_tokenize(sentence)
    lowered = [token.lower() for token in tokens]
    lemmas = []
    for token in lowered:
        if re.search('[а-яА-ЯёЁ]', token):
            lemma = morph.parse(token)[0].normal_form
            lemmas.append(lemma)
    cleaned = [token for token in lemmas if re.fullmatch(r'[а-яё-]+', token) and token not in russian_stopwords]
    return cleaned

train_tokens = train_df['ru'].apply(preprocess_sentence)
train_tokens.head()


0                                                   []
1                                                   []
2    [приводить, ниже, расписание, являться, предва...
3    [вверять, удостовериться, шотландец, приуменьш...
4                      [тусоваться, несколько, неделя]
Name: ru, dtype: object

In [37]:
test_tokens = test_df['ru'].apply(preprocess_sentence)

## TF-IDF векторизация
Обучаем TF-IDF на обучающих предложениях.

In [38]:
train_corpus = train_tokens.apply(lambda tokens: ' '.join(tokens))
test_corpus = test_tokens.apply(lambda tokens: ' '.join(tokens))
tfidf = TfidfVectorizer()
tfidf_matrix_train = tfidf.fit_transform(train_corpus)
tfidf_matrix_test = tfidf.transform(test_corpus)
tfidf_matrix_train.shape

(200000, 57540)

## Поиск ближайших предложений (TF-IDF)
Находим топ-3 похожих обучающих предложений для нескольких тестовых примеров.

In [39]:
def top_k_similar(vector, matrix, k=3):
    similarities = cosine_similarity(vector, matrix)[0]
    top_idx = similarities.argsort()[::-1][:k]
    return list(zip(top_idx, similarities[top_idx]))

for i in range(5):
    vector = tfidf_matrix_test[i]
    matches = top_k_similar(vector, tfidf_matrix_train)
    sentence = test_df.loc[i, 'ru']
    print(f'Тестовое предложение: {sentence}')
    print('Топ-3 похожих предложения из тренирочного корпуса:')
    for idx, score in matches:
        ref_sentence = train_df.loc[idx, 'ru']
        print(f'  score={score:.3f} | {ref_sentence}')
    print()

Тестовое предложение: Только бы не вылететь.
Топ-3 похожих предложения из тренирочного корпуса:
  score=0.745 | Виктор так и не вылетел.
  score=0.695 | - Я мог бы вылететь сегодня вечером.
  score=0.676 | Шпилька вылетела

Тестовое предложение: И как ты только справляешься, папа, таская эти коробки взад-вперед целый день.
Топ-3 похожих предложения из тренирочного корпуса:
  score=0.413 | их целая коробка стоит!
  score=0.373 | Целый день?
  score=0.368 | Ты справляешься?

Тестовое предложение: Возможно, у нас есть небольшое преимущество в переговорах.
Топ-3 похожих предложения из тренирочного корпуса:
  score=0.509 | Небольшая.
  score=0.509 | Да, да, после небольшого...
  score=0.509 | Она совсем небольшая.

Тестовое предложение: Сколько времени вы будете делать то, что ему нужно?
Топ-3 похожих предложения из тренирочного корпуса:
  score=0.729 | - Сколько у нас времени?
  score=0.729 | Сколько времени?
  score=0.729 | Сколько у нас есть времени?

Тестовое предложение: 1 апреля Прези

## Обучение модели Word2Vec
Строим векторные представления слов на основе обучающего корпуса.

In [ ]:
w2v_model = Word2Vec(sentences=train_tokens.tolist(), vector_size=300, window=5, min_count=1, workers=10, seed=42)
w2v_model.wv.key_to_index.__len__()

59645

## Средние вектора предложений (Word2Vec)
Представляем каждое предложение как средний вектор его слов.

In [41]:
def sentence_vector(tokens, model):
    vectors = []
    for token in tokens:
        if token in model.wv:
            vectors.append(model.wv[token])
    if vectors:
        return np.mean(vectors, axis=0)
    return np.zeros(model.vector_size)

train_vectors = np.vstack([sentence_vector(tokens, w2v_model) for tokens in train_tokens])
test_vectors = np.vstack([sentence_vector(tokens, w2v_model) for tokens in test_tokens])
train_vectors.shape

(200000, 100)

## Поиск ближайших предложений (Word2Vec)
Снова находим топ-3 похожих примера, но уже с помощью средних векторов.

In [43]:
for i in range(5):
    vector = test_vectors[i].reshape(1, -1)
    matches = top_k_similar(vector, train_vectors)
    sentence = test_df.loc[i, 'ru']
    print(f'Тестовое предложение: {sentence}')
    print('Топ-3 похожих предложения:')
    for idx, score in matches:
        ref_sentence = train_df.loc[idx, 'ru']
        print(f'  score={score:.3f} | {ref_sentence}')
    print()

Тестовое предложение: Только бы не вылететь.
Топ-3 похожих предложения:
  score=0.998 | Они вылетели из Барроу.
  score=0.995 | Шпилька вылетела
  score=0.991 | Виктор так и не вылетел.

Тестовое предложение: И как ты только справляешься, папа, таская эти коробки взад-вперед целый день.
Топ-3 похожих предложения:
  score=0.992 | Ольга, мне кажется, что будет неплохо, если ты поживешь пару дней у нас с Элизабет.
  score=0.989 | С сегодняшнего дня для перепихона далеко ехать не придётся.
  score=0.989 | Счастливого дня благодарения, доктор Кастеллано.

Тестовое предложение: Возможно, у нас есть небольшое преимущество в переговорах.
Топ-3 похожих предложения:
  score=0.959 | Я решил предложить им одно из самых желанных лондонских владений.
  score=0.954 | Чтобы найти оптимальный баланс, можно сделать серию снимков с различными значениями выдержки и экспозиции вспышки.
  score=0.952 | www.mebeam.com - Здесь вы можете создать пространство и все, кто пришел в это пространство может быть видн

## Вывод: Визуально TF-IDF справился лучше чем Word2Vec.